---
# **2. Расчёт логарифмических доходностей и статистика**

Логарифмические доходности обладают удобными свойствами для стохастического моделирования:
- Аддитивны: $\ln\frac{P(T)}{P(0)} = \sum_{t} \ln\frac{P(t)}{P(t-1)}$
- Инвариантны к сплитам акций
- Примерно нормально распределены

Логарифмическая доходность актива $i$ в день $t$:
$$r_i(t) = \ln\left(\frac{P_i(t)}{P_i(t-1)}\right)$$

Матрица дневных доходностей:
$$\mathbf{R} = \ln(\mathbf{P}_{t}) - \ln(\mathbf{P}_{t-1}), \quad \mathbf{R} \in \mathbb{R}^{(T-1) \times n}$$

Если $|r_i(t)| > \tau$ (обычно $\tau = 0.70$), предполагается сплит акции.
Процедура: заменить на NaN и использовать forward fill.

$$\mu_i^{\text{annual}} = \mu_i \times 252, \quad \sigma_i^{\text{annual}} = \sigma_i \times \sqrt{252}$$

где 252 — торговые дни в году.

$$\Sigma = \mathbb{E}[(\mathbf{r} - \boldsymbol{\mu})(\mathbf{r} - \boldsymbol{\mu})^T]$$

Аннуализированная:
$$\Sigma^{\text{annual}} = \Sigma \times 252$$

- **Собственные значения** ($\lambda_{\min}, \lambda_{\max}$) характеризуют спектр
- **Число обусловленности**: $\kappa = \lambda_{\max} / \lambda_{\min}$
- Если $\kappa > 100$ → высокая мультиколлинеарность

**Эксцесс**:
$$\text{Эксцесс}(r) = \frac{\mathbb{E}[(r-\mu)^4]}{\sigma^4} - 3$$
- Эксцесс = 0 → нормальное распределение
- Эксцесс > 0 → тяжелые хвосты (экстремальные события)

**Асимметрия**:
$$\text{Асимметрия}(r) = \frac{\mathbb{E}[(r-\mu)^3]}{\sigma^3}$$

---

In [ ]:
returns_raw = np.log(priced_df / priced_df.shift(1)).dropna()

print(f"Исходные доходности: {returns_raw.shape[0]} дней × {returns_raw.shape[1]} активов")

print("\n Детектор сплитов акций (обнаружение экстремальных доходностей):\n")

returns = returns_raw.copy()
split_threshold = 0.70

splits_detected = {}

for ticker in returns.columns:
    extreme_days = np.abs(returns[ticker]) > split_threshold

    if extreme_days.any():
        extreme_returns = returns.loc[extreme_days, ticker]
        splits_detected[ticker] = extreme_returns

        print(f" {ticker}: обнаружены потенциальные сплиты:")
        for date, ret in extreme_returns.items():
            ratio = np.exp(ret)
            direction = "обратный" if ret < 0 else "прямой"
            print(f"   {date.date()}: {ret*100:>8.2f}% ({direction} сплит {ratio:.2f}x)")

        returns.loc[extreme_days, ticker] = np.nan

returns = returns.ffill()
returns = returns.dropna()

print(f"\n После обработки: {returns.shape[0]} дней × {returns.shape[1]} активов")

actual_trading_days = len(returns)

calendar_days = (priced_df.index[-1] - priced_df.index[0]).days
trading_days_per_year = int(actual_trading_days * 365 / max(calendar_days, 1))

TRADING_DAYS = 252

print(f" Статистика торговых дней:")
print(f"   Период данных: {priced_df.index[0].date()} → {priced_df.index[-1].date()}")
print(f"   Календарных дней: {calendar_days}")
print(f"   Торговых дней в выборке: {actual_trading_days}")
print(f"   Расчётное среднее торговых дней в год: {trading_days_per_year}")
print(f"   Используется для аннуализации: {TRADING_DAYS} дней (стандарт)")

cov_matrix_daily = returns.cov()

cov_matrix = cov_matrix_daily * TRADING_DAYS

correlation_matrix = returns.corr()

eigenvalues = np.linalg.eigvalsh(cov_matrix.values)
min_eig = eigenvalues.min()
max_eig = eigenvalues.max()

print(f"\n Спектр матрицы ковариации:")
print(f"   λ_min (наименьшее собственное значение): {min_eig:.2e}")
print(f"   λ_max (наибольшее собственное значение): {max_eig:.2e}")
print(f"   Число обусловленности (condition number): {max_eig / min_eig:.2e}")

if min_eig > 1e-10:
    print(f"\n Матрица положительно определена")
else:
    print(f"\n Матрица близка к вырожденной")
    print("   Возможные причины:")
    print("   - Недостаточно данных (T < n_assets)")
    print("   - Высокая мультиколлинеарность")
    print("   - Ошибки при расчете ковариации")
    print("\n   ➜ Рекомендация: использовать Ledoit-Wolf shrinkage")

    try:
        from sklearn.covariance import LedoitWolf
        lw = LedoitWolf()
        cov_matrix_shrunk = lw.fit(returns.values).covariance_
        cov_matrix = pd.DataFrame(
            cov_matrix_shrunk,
            index=cov_matrix.index,
            columns=cov_matrix.columns
        )

        eigenvalues_shrunk = np.linalg.eigvalsh(cov_matrix.values)
        print(f"   λ_min после shrinkage: {eigenvalues_shrunk.min():.2e}")
    except ImportError:
        print("\n sklearn не установлен, применяю базовую регуляризацию")
        cov_matrix = cov_matrix + np.eye(len(cov_matrix)) * abs(min_eig) * 1.1

mu = returns.mean() * TRADING_DAYS

annual_vol = returns.std() * np.sqrt(TRADING_DAYS)

print(f"\n Ожидаемые годовые доходности (после очистки):\n")
for ticker in mu.sort_values(ascending=False).index:
    emoji = "🟢" if mu[ticker] > 0 else "🔴"
    print(f"   {emoji} {ticker:6s}: μ = {mu[ticker]*100:>7.2f}% | σ = {annual_vol[ticker]*100:>7.2f}%")

corr_values = correlation_matrix.values
upper_tri = np.triu_indices(len(corr_values), k=1)
high_corr_mask = np.abs(corr_values[upper_tri]) > 0.95

if high_corr_mask.any():
    print(f"\n Высокие корреляции (|ρ| > 0.95) — возможна мультиколлинеарность:")
    tickers = correlation_matrix.columns.tolist()
    for idx in np.where(high_corr_mask)[0]:
        i, j = upper_tri[0][idx], upper_tri[1][idx]
        print(f"   {tickers[i]:6s} ↔ {tickers[j]:6s}: ρ = {corr_values[i, j]:.4f}")
else:
    print(f"\n Мультиколлинеарности не обнаружено (все |ρ| < 0.95)")

kurtosis_values = returns.kurtosis()
skewness_values = returns.skew()

print(f" Статистика распределений доходностей:\n")
print(f"   Активы отсортированы по эксцессу (убывающий):\n")

for ticker in kurtosis_values.sort_values(ascending=False).index:
    kurt = kurtosis_values[ticker]
    skew = skewness_values[ticker]

    if kurt > 5:
        kurt_level = "очень высокий (тяжёлые хвосты)"
    elif kurt > 3.5:
        kurt_level = "высокий (тяжелее, чем нормальное)"
    elif kurt > 2.5:
        kurt_level = "нормальный"
    else:
        kurt_level = "низкий (лёгкие хвосты)"

    if abs(skew) < 0.1:
        skew_level = "симметричное"
    elif skew > 0.5:
        skew_level = "положительный перекос (правый хвост)"
    elif skew < -0.5:
        skew_level = "отрицательный перекос (левый хвост)"
    else:
        skew_level = "слабый перекос"

    print(f"   {ticker:6s}:")
    print(f"      Эксцесс: {kurt:>7.3f} — {kurt_level}")
    print(f"      Асимметрия: {skew:>7.3f} — {skew_level}")

mean_kurtosis = kurtosis_values.mean()
mean_skewness = skewness_values.mean()

print(f"\n   Средние по портфелю:")
print(f"      Средний эксцесс: {mean_kurtosis:.3f}")
print(f"      Средняя асимметрия: {mean_skewness:.3f}")

if mean_kurtosis > 3:
    print(f"\n    ВАЖНО: Портфель имеет ТЯЖЁЛЫЕ ХВОСТЫ (эксцесс > 3)")
    print(f"      → Риск экстремальных движений выше, чем предсказывает нормальное распределение")
    print(f"      → При оценке VaR и стресс-тестах учитывать вероятность больших убытков")

if abs(mean_skewness) > 0.2:
    if mean_skewness > 0:
        print(f"\n     Портфель имеет положительный ПЕРЕКОС (асимметрия > 0)")
        print(f"      → Больше частых малых убытков и реже большие прибыли")
    else:
        print(f"\n     Портфель имеет отрицательный ПЕРЕКОС (асимметрия < 0)")
        print(f"      → Больше частых малых прибылей и реже большие убытки")


print("\n Диагностика выбросов:\n")
extreme_threshold = 3
extreme_days_before = (np.abs(returns_raw) > extreme_threshold * returns_raw.std()).any(axis=1).sum()
extreme_days_after = (np.abs(returns) > extreme_threshold * returns.std()).any(axis=1).sum()

print(f"   До очистки сплитов: {extreme_days_before} дней с доходностями > 3σ")
print(f"   После очистки сплитов: {extreme_days_after} дней с доходностями > 3σ")

fig, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, ax=ax,
            cbar_kws={'label': 'Корреляция'})
ax.set_title('Корреляционная матрица активов', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig('correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()

n_assets = len(asset_names)
n_cols = 5
n_rows = int(np.ceil(n_assets / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, n_rows * 4))
axes = axes.flatten()

for idx, ticker in enumerate(asset_names):
    ax = axes[idx]

    data = returns[ticker]
    mean_ret = data.mean()
    std_ret = data.std()

    ax.hist(data, bins=40, density=True, alpha=0.7, color='steelblue', edgecolor='black')

    x = np.linspace(data.min(), data.max(), 100)
    from scipy.stats import norm
    ax.plot(x, norm.pdf(x, mean_ret, std_ret), 'r-', linewidth=2, label='Normal fit')

    kurt = data.kurtosis()
    skew = data.skew()
    ax.set_title(f'{ticker}\nKurt={kurt:.2f}, Skew={skew:.2f}', fontweight='bold', fontsize=10)
    ax.set_xlabel('Доходность')
    ax.set_ylabel('Плотность вероятности')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('returns_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close()

print(f" Переменные:")
print(f"   returns            : {returns.shape} — лог-доходности (очищенные)")
print(f"   cov_matrix         : {cov_matrix.shape} — годовая ковариация")
print(f"   correlation_matrix : {correlation_matrix.shape} — корреляции")
print(f"   mu                 : ({len(mu)},) — ожидаемые годовые доходности")
print(f"   annual_vol         : ({len(annual_vol)},) — годовая волатильность")
print(f"   asset_names        : {asset_names}")
print(f"\n Статистика:")
print(f"   Средняя доходность: {mu.mean()*100:.2f}%")
print(f"   Средняя волатильность: {annual_vol.mean()*100:.2f}%")
print(f"   Средняя корреляция: {correlation_matrix.values[upper_tri].mean():.4f}")
print(f"   Средний эксцесс: {kurtosis_values.mean():.3f}")
print(f"   Средняя асимметрия: {skewness_values.mean():.3f}")